# Important libraries

In [6]:
import win32com.client
import sys
import time
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Openserver Setup

Openserver class with methods to connect to the server, send commands, and disconnect from the server.

In [24]:
class OpenServer():
    "Class for holding ActiveX reference. Allows license disconnection"
    def __init__(self):
        self.status = "Disconnected"
        self.OSReference = None
    
    def Connect(self):
        self.OSReference = win32com.client.Dispatch("PX32.OpenServer.1")
        self.status = "Connected"
        print("OpenServer connected")
        
    def Disconnect(self):
        self.OSReference = None
        self.status = "Disconnected"
        print("OpenServer disconnected")

In [25]:
def GetAppName(sv):
    # function for returning app name from tag string
    pos = sv.find(".")
    if pos < 2:
        sys.exit("GetAppName: Badly formed tag string")
    app_name = sv[:pos]
    if app_name.lower() not in ["prosper", "mbal", "gap", "pvt", "resolve",
                                   "reveal"]:
        sys.exit("GetAppName: Unrecognised application name in tag string")
    return app_name


def DoCmd(OpenServe, cmd):
    # perform a command and check for errors
    lerr = OpenServe.OSReference.DoCommand(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoCmd: " + err)


def DoSet(OpenServe, sv, val):
    # set a value and check for errors
    lerr = OpenServe.OSReference.SetValue(sv, val)
    app_name = GetAppName(sv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSet: " + err)
    
def DoGet(OpenServe, gv):
    # get a value and check for errors
    get_value = OpenServe.OSReference.GetValue(gv)
    app_name = GetAppName(gv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGet: " + err)
    return get_value


def DoSlowCmd(OpenServe, cmd):
    # perform a command then wait for command to exit and check for errors
    step = 0.001
    app_name = GetAppName(cmd)
    lerr = OpenServe.OSReference.DoCommandAsync(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)
    while OpenServe.OSReference.IsBusy(app_name) > 0:
        if step < 2:
            step = step*2
        time.sleep(step)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)


def DoGAPFunc(OpenServe, gv):
    DoSlowCmd(gv)
    DoGAPFunc = DoGet(OpenServe, "GAP.LASTCMDRET")
    lerr = OpenServe.OSReference.GetLastError("GAP")
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGAPFunc: " + err)
    return DoGAPFunc


def OSOpenFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.OPENFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSOpenFile: " + err)


def OSSaveFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.SAVEFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSSaveFile: " + err)

# Multiple well calculation

In [4]:
#gas availability and increment steps
number_of_wells = int(input('Please enter the number of wells:'))
gas_available = float(input('Please enter available gas'))
increment = float(input('Please enter the increment value'))
gas_injection_array = np.arange(0, gas_available, increment)
df=pd.DataFrame()
df['gas_injection_array'] = gas_injection_array
df

,gas_injection_array
0,0.0
1,0.1
2,0.2
3,0.3
4,0.4
5,0.5
6,0.6
7,0.7
8,0.8
9,0.9


In [5]:
# Initialises an 'OpenServer' class

petex = OpenServer()

# Creates ActiveX reference and holds a license

petex.Connect()
oil_rates_for_wells = {}

# Perform functions

cwd = os.getcwd()
for j in range(1,number_of_wells+1):
    #opening well file
    OSOpenFile(petex, cwd + f'\models\well_{j}.Out', 'PROSPER')
    print(f'Well {j} opened')
    #oil rates calculation
    for i in range(0,len(gas_injection_array)):
        command = f'PROSPER.ANL.SYS.Sens.SensDB.Sens[138].Vals[{i}]'
        DoSet(petex, command,gas_injection_array[i])
    DoCmd(petex, 'PROSPER.ANL.SYS.CALC')
    oil_rates = [] #list of oil rates for 1 well
    for i in range(0, len(gas_injection_array)):
        value = f'PROSPER.OUT.SYS.Results[{i}].Sol.OilRate'
        oil_rates.append(np.round(float(DoGet(petex, value)),2))
    oil_rates_for_wells[f'well_{j}'] = oil_rates
    #closing file   
    OSSaveFile(petex, cwd + f'\well_{j}.Out', 'PROSPER')
    print(f'Well {j} closed')

#merging to main dataframe
for i in range(1,number_of_wells+1):
    df[f'Well_{i}'] = oil_rates_for_wells[f'well_{i}']

df.head()

OpenServer connected
Well 1 opened
Well 1 closed
Well 2 opened
Well 2 closed
Well 3 opened
Well 3 closed
Well 4 opened
Well 4 closed
Well 5 opened
Well 5 closed
Well 6 opened
Well 6 closed
Well 7 opened
Well 7 closed
Well 8 opened
Well 8 closed
Well 9 opened
Well 9 closed
Well 10 opened
Well 10 closed


,gas_injection_array,Well_1,Well_2,Well_3,Well_4,Well_5,Well_6,Well_7,Well_8,Well_9,Well_10
0,0.0,878.37,1657.35,1576.44,1362.42,997.05,1858.40,1767.62,1123.93,1717.52,1749.18
1,0.1,896.05,1668.40,1588.09,1375.90,1013.68,1867.23,1777.72,1139.83,1728.00,1759.44
2,0.2,911.90,1677.71,1597.90,1387.41,1027.60,1874.69,1786.06,1153.42,1739.01,1767.97
3,0.3,924.19,1685.58,1606.21,1397.28,1041.02,1880.97,1793.10,1164.63,1746.45,1775.18
4,0.4,935.19,1692.24,1613.28,1405.79,1051.22,1886.24,1799.04,1174.25,1752.74,1781.27


In [6]:
fig = go.Figure()
for i in range(1,number_of_wells+1):
    
    fig.add_trace(go.Scatter(x=df['gas_injection_array'], y=df[f'Well_{i}'],\
                            mode='lines', name=f'Well_{i}'))
fig.update_layout(title='Sensitivity Plot',
                   xaxis_title='Gaslift Gas Injection Rate(MMscf/day)',
                   yaxis_title='Oil Rate(STB/day)',
                   title_x=0.5, 
                   )
fig.update_xaxes(nticks=50)
fig.update_yaxes(nticks=20)
fig.show()

In [7]:
df.head()

,gas_injection_array,Well_1,Well_2,Well_3,Well_4,Well_5,Well_6,Well_7,Well_8,Well_9,Well_10
0,0.0,878.37,1657.35,1576.44,1362.42,997.05,1858.40,1767.62,1123.93,1717.52,1749.18
1,0.1,896.05,1668.40,1588.09,1375.90,1013.68,1867.23,1777.72,1139.83,1728.00,1759.44
2,0.2,911.90,1677.71,1597.90,1387.41,1027.60,1874.69,1786.06,1153.42,1739.01,1767.97
3,0.3,924.19,1685.58,1606.21,1397.28,1041.02,1880.97,1793.10,1164.63,1746.45,1775.18
4,0.4,935.19,1692.24,1613.28,1405.79,1051.22,1886.24,1799.04,1174.25,1752.74,1781.27


In [8]:
#extracting max gas lift oil rate pairs
df_new = df.drop('gas_injection_array', axis=1)
wells = []
i=1
for element in df_new.idxmax():
    oil_rate = df_new[f'Well_{i}'][element]
    gas_lift = df['gas_injection_array'][element]
    wells.append({'production_rate':oil_rate, 'gas_lift':gas_lift})
    i+=1
wells

[{'production_rate': 987.44, 'gas_lift': 1.7000000000000002},
 {'production_rate': 1717.55, 'gas_lift': 1.4000000000000001},
 {'production_rate': 1643.5, 'gas_lift': 1.5},
 {'production_rate': 1448.2, 'gas_lift': 1.7000000000000002},
 {'production_rate': 1104.21, 'gas_lift': 1.9000000000000001},
 {'production_rate': 1904.17, 'gas_lift': 1.2000000000000002},
 {'production_rate': 1819.56, 'gas_lift': 1.3},
 {'production_rate': 1225.93, 'gas_lift': 1.9000000000000001},
 {'production_rate': 1775.91, 'gas_lift': 1.3},
 {'production_rate': 1802.58, 'gas_lift': 1.3}]

In [1]:
wells = [{'production_rate': 987.44, 'gas_lift': 1.7},
         {'production_rate': 1717.55, 'gas_lift': 1.4},
         {'production_rate': 1643.5, 'gas_lift': 1.5},
         {'production_rate': 1448.2, 'gas_lift': 1.7},
         {'production_rate': 1104.21, 'gas_lift': 1.9},
         {'production_rate': 1904.17, 'gas_lift': 1.2},
         {'production_rate': 1819.56, 'gas_lift': 1.3},
         {'production_rate': 1225.93, 'gas_lift': 1.9},
         {'production_rate': 1775.91, 'gas_lift': 1.3},
         {'production_rate': 1802.58, 'gas_lift': 1.3}]
well_list = [(d['production_rate'], d['gas_lift']) for d in wells]

# Ant Colony Algorithm

In [28]:
import numpy as np

def optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count):
    """
    Optimizes the allocation of gas to a group of wells using an ant colony algorithm.
    
    Parameters:
    - wells: a list of dictionaries representing the wells, where each dictionary contains the following keys:
        - 'production_rate': the production rate of the well (in STB/day)
        - 'gas_lift': the gas lift of the well (in STB/day)
    - gas_capacity: the total gas capacity available for allocation (in STB/day)
    - alpha: the weight of the pheromone trail in the decision making process
    - beta: the weight of the heuristic value in the decision making process
    - evaporation_rate: the rate at which the pheromone trails evaporate
    - iteration_count: the number of iterations to run the algorithm for
    
    Returns:
    - A list of dictionaries representing the optimized allocation of gas to the wells, where each dictionary contains the following keys:
        - 'well_id': the ID of the well
        - 'allocation': the optimized allocation of gas to the well (in STB/day)
    """
    # Initialize the pheromone trails and heuristic values for each well
    pheromone_trails = np.ones(len(wells))
    heuristic_values = np.array([well['gas_lift'] / well['production_rate'] for well in wells])
    
    # Run the algorithm for the specified number of iterations
    for i in range(iteration_count):
        # Initialize the allocation for each well
        allocations = np.zeros(len(wells))
        
        # Allocate gas to each well using the ant colony algorithm
        remaining_gas = gas_capacity
        for j, well in enumerate(wells):
            # Calculate the allocation probability for the well
            allocation_probability = (pheromone_trails[j] ** alpha) * (heuristic_values[j] ** beta)
            allocation_probability /= np.sum((pheromone_trails ** alpha) * (heuristic_values ** beta))
            
            # Allocate gas to the well based on the allocation probability
            allocation = remaining_gas * allocation_probability
            allocations[j] = allocation
            remaining_gas -= allocation
            
        # Evaporate the pheromone trails based on the evaporation rate
        pheromone_trails *= (1 - evaporation_rate)
        
        # Update the pheromone trails based on the allocations
        for j, well in enumerate(wells):
            pheromone_trails[j] += allocations[j] / well['production_rate']
    
    # Return the optimized allocation of gas to the wells
    return [{'well_id': j, 'allocation': allocations[j]} for j in range(len(wells))]
   


# Current solution

In [58]:
import numpy as np

def optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count):
    """
    Optimizes the allocation of gas to a group of wells using an ant colony algorithm.
    
    Parameters:
    - wells: a list of dictionaries representing the wells, where each dictionary contains the following keys:
        - 'production_rate': the production rate of the well (in STB/day)
        - 'gas_lift': the gas lift of the well (in STB/day)
    - gas_capacity: the total gas capacity available for allocation (in STB/day)
    - alpha: the weight of the pheromone trail in the decision making process
    - beta: the weight of the heuristic value in the decision making process
    - evaporation_rate: the rate at which the pheromone trails evaporate
    - iteration_count: the number of iterations to run the algorithm for
    
    Returns:
    - A list of dictionaries representing the optimized allocation of gas to the wells, where each dictionary contains the following keys:
        - 'well_id': the ID of the well
        - 'allocation': the optimized allocation of gas to the well (in STB/day)
    """
    # Initialize the pheromone trails and heuristic values for each well
    pheromone_trails = np.ones(len(wells))
    heuristic_values = np.array([well['gas_lift'] / well['production_rate'] for well in wells])
    
    # Run the algorithm for the specified number of iterations
    for i in range(iteration_count):
        # Initialize the allocation for each well
        allocations = np.zeros(len(wells))
        
        # Allocate gas to each well using the ant colony algorithm
        remaining_gas = gas_capacity
        for j, well in enumerate(wells):
            # Calculate the allocation probability for the well
            allocation_probability = (pheromone_trails[j] ** alpha) * (heuristic_values[j] ** beta)
            allocation_probability /= np.sum((pheromone_trails ** alpha) * (heuristic_values ** beta))
            
            # Allocate gas to the well based on the allocation probability
            allocation = remaining_gas * allocation_probability
            allocations[j] = allocation
            remaining_gas -= allocation
            
        # Evaporate the pheromone trails based on the evaporation rate
        pheromone_trails *= (1 - evaporation_rate)
        
        # Update the pheromone trails based on the allocations
        for j, well in enumerate(wells):
            pheromone_trails[j] += allocations[j] / well['production_rate']
    
    # Return the optimized allocation of gas to the wells
    return [{'well_id': j, 'allocation': allocations[j]} for j in range(len(wells))]
   


In [2]:
import numpy as np

def optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count):
    """
    Optimizes the allocation of gas to a group of wells using an ant colony algorithm.
    
    Parameters:
    - wells: a list of dictionaries representing the wells, where each dictionary contains the following keys:
        - 'production_rate': the production rate of the well (in STB/day)
        - 'gas_lift': the gas lift of the well (in STB/day)
    - gas_capacity: the total gas capacity available for allocation (in STB/day)
    - alpha: the weight of the pheromone trail in the decision making process
    - beta: the weight of the heuristic value in the decision making process
    - evaporation_rate: the rate at which the pheromone trails evaporate
    - iteration_count: the number of iterations to run the algorithm for
    
    Returns:
    - A list of dictionaries representing the optimized allocation of gas to the wells, where each dictionary contains the following keys:
        - 'well_id': the ID of the well
        - 'allocation': the optimized allocation of gas to the well (in STB/day)
    """
    # Initialize the pheromone trails and heuristic values for each well
    pheromone_trails = np.ones(len(wells))
    heuristic_values = np.array([well['production_rate'] / well['gas_lift'] for well in wells])
    
    # Run the algorithm for the specified number of iterations
    for i in range(iteration_count):
        # Initialize the allocation for each well
        allocations = np.zeros(len(wells))
        
        # Allocate gas to each well using the ant colony algorithm
        remaining_gas = gas_capacity
        for j, well in enumerate(wells):
            # Calculate the allocation probability for the well
            allocation_probability = (pheromone_trails[j] ** alpha) * (heuristic_values[j] ** beta)
            allocation_probability /= np.sum((pheromone_trails ** alpha) * (heuristic_values ** beta))
            
            # Allocate gas to the well based on the allocation probability
            allocation = min(well['gas_lift'], remaining_gas * allocation_probability)
            allocations[j] = allocation
            remaining_gas -= allocation
        
        # Evaporate the pheromone trails based on the evaporation rate
        pheromone_trails *= (1 - evaporation_rate)
        
        # Update the pheromone trails based on the allocations
        for j, well in enumerate(wells):
            pheromone_trails[j] += allocations[j] / well['production_rate']
    
    # Return the optimized allocation of gas to the wells
    return [{'well_id': j, 'allocation': allocations[j]} for j in range(len(wells))]


# Testing the current solution

In [3]:
gas_capacity = 3.5
alpha = 1
beta = 2
evaporation_rate = 0.1
iteration_count = 30

# Optimize the allocation of gas to the wells
allocations = optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count)

# Print the optimized allocation
print(allocations)


[{'well_id': 0, 'allocation': 0.09094789272866953}, {'well_id': 1, 'allocation': 0.40633464193362706}, {'well_id': 2, 'allocation': 0.2819790109739494}, {'well_id': 3, 'allocation': 0.15247020108331652}, {'well_id': 4, 'allocation': 0.06638461126992312}, {'well_id': 5, 'allocation': 0.5013861782060725}, {'well_id': 6, 'allocation': 0.3063987178399005}, {'well_id': 7, 'allocation': 0.05379974424759531}, {'well_id': 8, 'allocation': 0.23771980390858372}, {'well_id': 9, 'allocation': 0.20867506038042455}]


In [4]:
sum_gas = 0
for alloc in allocations:
    sum_gas+=alloc['allocation']
print(sum_gas)

2.3060958625720622


# Visualize the current solution

In [27]:
# for j in range(len(allocations)):
gas_dict = {}
for i in range(len(allocations)):
    gas_available = allocations[i]['allocation']
    increment = 0.05
    gas_injection_array = np.arange(0, gas_available, increment)
    gas_dict[f'gas_injection_array_{i+1}'] = gas_injection_array
gas_dict

{'gas_injection_array_1': array([0.  , 0.05]),
 'gas_injection_array_2': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 ]),
 'gas_injection_array_3': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25]),
 'gas_injection_array_4': array([0.  , 0.05, 0.1 , 0.15]),
 'gas_injection_array_5': array([0.  , 0.05]),
 'gas_injection_array_6': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ]),
 'gas_injection_array_7': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 ]),
 'gas_injection_array_8': array([0.  , 0.05]),
 'gas_injection_array_9': array([0.  , 0.05, 0.1 , 0.15, 0.2 ]),
 'gas_injection_array_10': array([0.  , 0.05, 0.1 , 0.15, 0.2 ])}

In [37]:
# Initialises an 'OpenServer' class

petex = OpenServer()

# Creates ActiveX reference and holds a license

petex.Connect()
oil_rates_for_wells = {}

# Perform functions

cwd = os.getcwd()
for j in range(len(gas_dict)):
#opening well file
    OSOpenFile(petex, cwd + f'\models\well_{j+1}.Out', 'PROSPER')
    print(f'Well {j+1} opened')

    for i in range(0,len(gas_dict[f'gas_injection_array_{j+1}'])):
            command = f'PROSPER.ANL.SYS.Sens.SensDB.Sens[138].Vals[{i}]'
            DoSet(petex, command,gas_dict[f'gas_injection_array_{j+1}'][i])
    DoCmd(petex, 'PROSPER.ANL.SYS.CALC')
    oil_rates = [] #list of oil rates for 1 well
    for i in range(0, len(gas_dict[f'gas_injection_array_{j+1}'])):
            value = f'PROSPER.OUT.SYS.Results[{i}].Sol.OilRate'
            oil_rates.append(np.round(float(DoGet(petex, value)),2))
    oil_rates_for_wells[f'well_{j+1}'] = oil_rates

    #closing file   
    OSSaveFile(petex, cwd + f'\well_{j+1}.Out', 'PROSPER')
    print(f'Well {j+1} closed')

#merging to main dataframe

# df[f'Well_2'] = oil_rates_for_wells[f'well_2']

oil_rates_for_wells

OpenServer connected
Well 1 opened
Well 1 closed
Well 2 opened
Well 2 closed
Well 3 opened
Well 3 closed
Well 4 opened
Well 4 closed
Well 5 opened
Well 5 closed
Well 6 opened
Well 6 closed
Well 7 opened
Well 7 closed
Well 8 opened
Well 8 closed
Well 9 opened
Well 9 closed
Well 10 opened
Well 10 closed


{'well_1': [878.37, 887.66],
 'well_2': [1657.35,
  1663.12,
  1668.4,
  1673.25,
  1677.71,
  1681.81,
  1685.58,
  1689.05,
  1692.24],
 'well_3': [1576.44, 1582.52, 1588.09, 1593.2, 1597.9, 1602.23],
 'well_4': [1362.42, 1369.43, 1375.9, 1381.88],
 'well_5': [997.05, 1005.79],
 'well_6': [1858.4,
  1863.0,
  1867.23,
  1871.12,
  1874.69,
  1877.97,
  1880.97,
  1883.72,
  1886.24,
  1888.53,
  1890.55],
 'well_7': [1767.62, 1772.88, 1777.72, 1782.06, 1786.06, 1789.73, 1793.1],
 'well_8': [1123.93, 1132.25],
 'well_9': [1717.52, 1723.02, 1728.0, 1734.79, 1739.01],
 'well_10': [1749.18, 1754.56, 1759.44, 1763.88, 1767.97]}

In [42]:
import plotly.express as px
import plotly.graph_objects as go

# Extract the keys (well names) and values (oil rates) from the oil_rates_for_wells dictionary
well_names = list(oil_rates_for_wells.keys())
oil_rates_values = list(oil_rates_for_wells.values())

# Extract the values (gas injection arrays) from the gas_dict dictionary
gas_injection_values = list(gas_dict.values())

# Create a figure with subplots for each well
fig = go.make_subplots(rows=len(well_names), cols=1, subplot_titles=well_names)

# Loop through the well names, oil rates, and gas injection arrays
for i, well_name, oil_rate, gas_injection in zip(range(len(well_names)), well_names, oil_rates_values, gas_injection_values):
    # Add a trace to the figure for the current well
    fig.add_trace(go.Scatter(x=gas_injection, y=oil_rate, name=well_name))

# Update the layout of the figure
fig.update_layout(title='Gas injection vs oil rate')

# Show the figure
fig.show()


AttributeError: module 'plotly.graph_objects' has no attribute 'make_subplots'

# 1

In [57]:
import numpy as np
class Ant:
  def __init__(self, num_wells):
    self.num_wells = num_wells
    self.solution = []
    
  def choose_next_node(self, pheromone, graph):
    # Choose the next node based on the pheromone levels and heuristic information
    pass
    
  def get_solution(self):
    return self.solution

def evaluate_solution(solution, wells, graph):
  total_oil_rate = 0
  total_gas_lift = 0
  for i, j in zip(solution[:-1], solution[1:]):
    oil_rate, _ = wells[i]
    gas_lift = graph[i, j]
    total_oil_rate += oil_rate * gas_lift
    total_gas_lift += gas_lift
  return total_oil_rate / total_gas_lift

def optimize_gas_lift(wells, gas_lift):
  num_wells = len(wells)
  
  # Initialize the pheromone levels on the edges
  pheromone = np.ones((num_wells, num_wells))
  
  # Initialize the graph with the initial gas lift allocation
  graph = np.zeros((num_wells, num_wells))
  for i in range(num_wells):
    _, gas_lift = wells[i]
    graph[i, i] = gas_lift
  
  # Create the colony of ants
  colony_size = 10  # Number of ants in the colony
  colony = []
  for _ in range(colony_size):
    colony.append(Ant(num_wells))
  
  # Iterate until the solution converges
  best_solution = None
  while True:
    # Let each ant choose the next node to visit based on the pheromone levels and heuristic information
    for ant in colony:
      ant.choose_next_node(pheromone, graph)
      
    # Update the pheromone levels based on the quality of the solutions found by the ants
    for ant in colony:
      solution = ant.get_solution()
      quality = evaluate_solution(solution)  # Calculate the quality of the solution
      for i, j in zip(solution[:-1], solution[1:]):
        pheromone[i, j] += quality
        pheromone[j, i] += quality
    


In [29]:
def test_optimize_gas_lift():
  # Test with a simple input where all wells have the same oil production rate
  wells = [(100, 1) for _ in range(10)]
  gas_lift = 5
  expected_output = [(100, 1.5) for _ in range(5)] + [(100, 1) for _ in range(5)]
  assert optimize_gas_lift(wells, gas_lift) == expected_output



In [32]:
  # Test with the list of tuples provided in the question
wells = [(987.44, 1.7), (1717.55, 1.4), (1643.5, 1.5), (1448.2, 1.7), (1104.21, 1.9),
          (1904.17, 1.2), (1819.56, 1.3), (1225.93, 1.9), (1775.91, 1.3), (1802.58, 1.3)]
gas_lift = 20
expected_output = [(987.44, 1.9), (1717.55, 1.6), (1643.5, 1.7), (1448.2, 1.9), (1104.21, 2.1),
                    (1904.17, 1.4), (1819.56, 1.5), (1225.93, 2.1), (1775.91, 1.5), (1802.58, 1.5)]
assert optimize_gas_lift(wells, gas_lift) == expected_output

TypeError: evaluate_solution() missing 2 required positional arguments: 'wells' and 'graph'

In [27]:
sum = 0
for i in range(len(allocations)):
    sum+=allocations[i]['allocation']
print(sum)

2.9280910888115694


# 2

In [23]:
def optimize_gas_lift(wells, gas_lift):
  # Sort the wells in descending order based on their current oil production rate
  wells.sort(key=lambda x: -x[0])
  
  # Allocate the gas lift to the wells
  for well in wells:
    oil_rate, current_gas_lift = well
    while gas_lift > 0:
      # Calculate the increase in oil production rate if we increase the gas lift by 1 unit
      increase_in_oil_rate = calculate_increase_in_oil_rate(oil_rate, current_gas_lift + 1)
      if increase_in_oil_rate <= 0:
        break
      # Increase the gas lift for this well and decrease the available gas lift
      current_gas_lift += 1
      gas_lift -= 1
      
  # Return the updated list of wells with the optimized gas lift allocation
  return [(oil_rate, current_gas_lift) for oil_rate, _ in wells]


In [25]:
# Test the ACO algorithm
wells = [{'production_rate': 987.44, 'gas_lift': 1.7000000000000002},
         {'production_rate': 1717.55, 'gas_lift': 1.4000000000000001},
         {'production_rate': 1643.5, 'gas_lift': 1.5},
         {'production_rate': 1448.2, 'gas_lift': 1.7000000000000002},
         {'production_rate': 1104.21, 'gas_lift': 1.9000000000000001},
         {'production_rate': 1904.17, 'gas_lift': 1.2000000000000002},
         {'production_rate': 1819.56, 'gas_lift': 1.3},
         {'production_rate': 1225.93, 'gas_lift': 1.9000000000000001},
         {'production_rate': 1775.91, 'gas_lift': 1.3},
         {'production_rate': 1802.58, 'gas_lift': 1.3}]
gas_lift = 20
num_ants = 10
num_iterations = 100

solution = ACO(wells, gas_lift, num_ants, num_iterations)
print(f"Solution: {solution}")
print(f"Total production rate: {calculate_production_rate(solution, wells)}")


ZeroDivisionError: division by zero

# 3

In [35]:
import random

# Define a class to represent a candidate solution (i.e., allocation of gas to the wells)
class Solution:
  def __init__(self, gas_allocation):
    self.gas_allocation = gas_allocation
    self.fitness = 0
  
  def evaluate_fitness(self, well_data):
    """Calculate the fitness of the solution by summing the production rates of all the wells.
    well_data is a list of tuples, where each tuple contains the maximum production rate and gas lift for a well.
    """
    total_production_rate = 0
    for i, (rate, lift) in enumerate(well_data):
      allocated_gas = self.gas_allocation[i]
      if allocated_gas > lift:
        allocated_gas = lift
      total_production_rate += rate * (allocated_gas / lift)
    self.fitness = total_production_rate
  
  def generate_offspring(self, max_gas):
    """Create a new solution by making small changes to the gas allocation of the current solution."""
    new_allocation = self.gas_allocation[:]  # create a copy of the gas allocation
    i = random.randint(0, len(new_allocation) - 1)  # choose a random well
    delta = random.uniform(-10, 10)  # choose a random change in gas allocation
    new_allocation[i] += delta
    if new_allocation[i] < 0:  # make sure the gas allocation is non-negative
      new_allocation[i] = 0
    # Make sure the total gas allocation does not exceed the maximum gas available
    if sum(new_allocation) > max_gas:
      new_allocation[i] -= (sum(new_allocation) - max_gas)
    return Solution(new_allocation)


# Define the main Ant Colony Optimization algorithm
def AntColonyOptimization(well_data, max_gas):
  # Initialize the algorithm by generating a random set of candidate solutions
  population = [Solution([random.uniform(0, max_gas) for _ in range(len(well_data))]) for _ in range(50)]
  for solution in population:
    solution.evaluate_fitness(well_data)
  
  # Run the algorithm for a specified number of iterations
  for _ in range(100):
    parent = random.choice(population)  # select a parent solution based on its fitness
    offspring = parent.generate_offspring(max_gas)  # generate an offspring solution
    offspring.evaluate_fitness(well_data)
    if offspring.fitness > parent.fitness:  # if the offspring is better than the parent, replace the parent with the offspring
      parent = offspring
  
  # Return the best solution found
  return max(population, key=lambda x: x.fitness)


# Example usage
well_data = [
  (100, 50),  # well 1 has a maximum production rate of 100 units with 50 units of gas lift
  (80, 40),   # well 2 has a maximum production rate of 80 units with 40 units of gas lift
  (60, 30),   # well 3 has a maximum production rate of 60 units with 30 units of gas lift
]
max_gas = 200  # total amount of gas available

best_solution = AntColonyOptimization(well_data, max_gas)
best_solution


# 4

In [52]:
import random

# Define a class to represent a candidate solution (i.e., allocation of gas to the wells)
class Solution:
  def __init__(self, gas_allocation):
    self.gas_allocation = gas_allocation
    self.fitness = 0
  
  def evaluate_fitness(self, well_data):
    """Calculate the fitness of the solution by summing the production rates of all the wells.
    well_data is a list of tuples, where each tuple contains the maximum production rate and gas lift for a well.
    """
    total_production_rate = 0
    for i, (rate, lift) in enumerate(well_data):
      allocated_gas = self.gas_allocation[i]
      if allocated_gas > lift:
        allocated_gas = lift
      total_production_rate += rate * (allocated_gas / lift)
    self.fitness = total_production_rate
  
  def generate_offspring(self, well_data):
      """Create a new solution by making small changes to the gas allocation of the current solution."""
      new_allocation = self.gas_allocation[:]  # create a copy of the gas allocation
      i = random.randint(0, len(new_allocation) - 1)  # choose a random well
      delta = random.uniform(-10, 10)  # choose a random change in gas allocation
      new_allocation[i] += delta
      if new_allocation[i] < 0:  # make sure the gas allocation is non-negative
        new_allocation[i] = 0
      # Make sure the gas allocation for each well does not exceed the maximum gas lift
      for i, (rate, lift) in enumerate(well_data):
        if new_allocation[i] > lift:
          new_allocation[i] = lift
      return Solution(new_allocation)



def AntColonyOptimization(well_data, max_gas):
  # Initialize the algorithm by generating a random set of candidate solutions
  population = [Solution([random.uniform(0, max_gas) for _ in range(len(well_data))]) for _ in range(50)]
  for solution in population:
    solution.evaluate_fitness(well_data)
  
  # Run the algorithm for a specified number of iterations
  for _ in range(100):
    parent = random.choice(population)  # select a parent solution based on its fitness
    offspring = parent.generate_offspring(well_data)  # generate an offspring solution
    offspring.evaluate_fitness(well_data)
    if offspring.fitness > parent.fitness:  # if the offspring is better than the parent, replace the parent with the offspring
      parent = offspring
  
  # Return the best solution found
  return max(population, key=lambda x: x.fitness)


# Print the results of the optimization
def print_results(solution, well_data):
  total_production_rate = 0
  total_gas_allocated = 0
  for i, (rate, lift) in enumerate(well_data):
    allocated_gas = solution.gas_allocation[i]
    if allocated_gas > lift:
      allocated_gas = lift
    production_rate = rate * (allocated_gas / lift)
    total_production_rate += production_rate
    total_gas_allocated += allocated_gas
    print(f"Well {i+1}: allocated {allocated_gas} units of gas, production rate = {production_rate}")
  print(f"Total gas allocated: {total_gas_allocated}")
  print(f"Total production rate: {total_production_rate}")



In [53]:
# Example usage
well_data = [
  (100, 50),  # well 1 has a maximum production rate of 100 units with 50 units of gas lift
  (80, 40),   # well 2 has a maximum production rate of 80 units with 40 units of gas lift
  (60, 30),  
  (70, 50),
  (80, 90),# well 3 has a maximum production rate of 60 units with 30 units of gas lift
]
max_gas = 200  # total amount of gas available

best_solution = AntColonyOptimization(well_data, max_gas)
print_results(best_solution, well_data)


Well 1: allocated 50 units of gas, production rate = 100.0
Well 2: allocated 40 units of gas, production rate = 80.0
Well 3: allocated 30 units of gas, production rate = 60.0
Well 4: allocated 50 units of gas, production rate = 70.0
Well 5: allocated 90 units of gas, production rate = 80.0
Total gas allocated: 260
Total production rate: 390.0


# 5

In [77]:
class Ant:
    def __init__(self):
        self.allocation = []
    
    def update_allocation(self, pheromone_levels):
        # Update the allocation of the ant based on the pheromone levels and the ACO rules
        pass

class ACO:
    def __init__(self, ants):
        self.ants = ants
    
    def update_allocations(self, pheromone_levels):
        for ant in self.ants:
            ant.update_allocation(pheromone_levels)

def calculate_pheromone_levels(allocations, costs, benefits):
    # Calculate the pheromone level for each allocation using a mathematical formula that takes into account the cost and benefit of the allocation
    pheromone_levels = []
    for allocation, cost, benefit in zip(allocations, costs, benefits):
        pheromone_level = benefit / cost
        pheromone_levels.append(pheromone_level)
    return pheromone_levels

def calculate_cost(allocation, costs):
    cost = 0
    for i, amount in enumerate(allocation):
        cost += amount * costs[i]
    return cost

def allocate_gas(costs, benefits, num_ants, num_iterations):
    # Initialize the list of ants and the ACO algorithm
    ants = [Ant() for _ in range(num_ants)]
    aco = ACO(ants)
    
    # Run the ACO algorithm for the specified number of iterations
    for i in range(num_iterations):
        pheromone_levels = calculate_pheromone_levels(ants, costs, benefits)
        aco.update_allocations(pheromone_levels)
    
    # Evaluate the allocations made by the ants and return the allocation with the lowest cost
    best_allocation = None
    best_cost = float('inf')
    for ant in ants:
        cost = calculate_cost(ant.allocation, costs)
        if cost < best_cost:
            best_allocation = ant.allocation
            best_cost = cost
    return best_allocation


In [78]:
# Define some test data
costs = [10, 20, 30]
benefits = [1, 2, 3]
num_ants = 3
num_iterations = 10

# Allocate the gas using the ACO algorithm
allocation = allocate_gas(costs, benefits, num_ants, num_iterations)
print(allocation)

[]


# 6

In [14]:
def objective_function(gas_allocation, flow_rate, pressure):
    """Calculate the total production from the wells.

    Parameters:
        gas_allocation (list): The amount of gas allocated to each well.
        flow_rate (list): The flow rate of each well.
        pressure (list): The gas lift pressure of each well.

    Returns:
        float: The total production from the wells.
    """
    total_production = 0
    for i, allocation in enumerate(gas_allocation):
        total_production += allocation * flow_rate[i] * pressure[i]
    return total_production


In [15]:
# Available gas supply
gas_supply = 1000  # in units of gas

# Flow rate of each well
well_flow_rates = [100, 200, 300, 400]  # in units of flow per unit time

# Cost of gas at different flow rates
cost_per_unit_gas = [1, 2, 3, 4]  # in units of cost per unit gas


In [16]:
# Import required modules
from pulp import LpVariable, LpProblem, LpMaximize

# Define the decision variables
gas_allocation = [LpVariable(f"gas_well_{i+1}", lowBound=0) for i in range(4)]


In [17]:
# Import required modules
from pulp import LpVariable, LpProblem, LpMaximize

# Define the decision variables
gas_allocation = [LpVariable(f"gas_well_{i+1}", lowBound=0) for i in range(4)]

# Define the optimization problem
prob = LpProblem("Gas Allocation Optimization", LpMaximize)

# Add the objective function
prob += sum(gas_allocation[i] * well_flow_rates[i]  for i in range(4))

# Add constraints
prob += sum(gas_allocation) <= gas_supply
for i in range(4):
    prob += gas_allocation[i] <= well_flow_rates[i]


In [18]:
import random

def ant_colony_optimization(prob, num_ants, num_iterations, evaporation_rate, choice_influence):
    """Solve the optimization problem using an ant colony algorithm.

    Parameters:
        prob (LpProblem): The optimization problem to solve.
        num_ants (int): The number of ants in the colony.
        num_iterations (int): The number of iterations to run the algorithm for.
        evaporation_rate (float): The rate at which the pheromone trails evaporate.
        choice_influence (float): The influence of the pheromone trails on the probability of an ant choosing a particular solution.

    Returns:
        list: The optimal solution found by the algorithm.
    """
    # Initialize pheromone trails
    pheromone_trails = [[1 for _ in prob.variables()] for _ in range(num_ants)]

    # Run the algorithm for the specified number of iterations
    for _ in range(num_iterations):
        # Create a list of solutions, one for each ant
        solutions = [[] for _ in range(num_ants)]

        # Iterate over each ant
        for i, ant in enumerate(solutions):
            # Iterate over each decision variable
            for var in prob.variables():
                # Calculate the probability of choosing this variable
                prob_choice = sum(pheromone_trails[i][j] ** choice_influence for j, _ in enumerate(prob.variables()))
                rand = random.uniform(0, 1)
                if rand < prob_choice:
                    # Choose this variable
                    ant.append(1)
                else:
                    # Don't choose this variable
                    ant.append(0)

        # Update the pheromone trails based on the solutions of the ants
        for i, ant in enumerate(solutions):
            for j, var in enumerate(prob.variables()):
                pheromone_trails[i][j] *= (1 - evaporation_rate)
                if ant[j] == 1 and prob.objective.value() is not None:
                    pheromone_trails[i][j] += 1/prob.objective.value()

    # Return the best solution found by the algorithm
    return max(solutions, key=lambda x: prob.objective.value())


In [19]:
# Solve the optimization problem using the ant colony algorithm
solution = ant_colony_optimization(prob, num_ants=100, num_iterations=1000, evaporation_rate=0.5, choice_influence=1)

# Print the optimal solution
print(solution)


TypeError: '>' not supported between instances of 'NoneType' and 'NoneType'

In [ ]:
# Print the optimal gas allocation
print("Optimal gas allocation:")
for i, var in enumerate(prob.variables()):
    print(f"Well {i+1}: {var.value()} units")

# Print the optimal objective value
print(f"Optimal objective value: {prob.objective.value()}")


Optimal gas allocation:
Well 1: None units
Well 2: None units
Well 3: None units
Well 4: None units
Optimal objective value: None
